<a href="https://colab.research.google.com/github/paperplane110/learning/blob/master/c5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning applied in CV
- Use CNN to process computer vision
- We will use Conv2D and MaxPooling2D layers

## 5.1 Warm up: use CNN on MNIST

In [25]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images,train_label),(test_images,test_label) = mnist.load_data()

- Process the data

In [26]:
train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32')/255

train_label = to_categorical(train_label)
test_label = to_categorical(test_label)

- Build the network

In [30]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28,28,1)))
network.add(layers.Dense(10, activation='softmax', input_shape=(512,)))

- Compile the Network

In [31]:
network.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

- Fit!

In [32]:
history = network.fit(train_images,train_label,epochs=5,batch_size=128)

Epoch 1/5


ValueError: ignored

- Evaluate

In [ ]:
test_loss, test_acc = network.evaluate(test_images,test_label)